In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision import transforms

class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.flatten(1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use a standard MNIST normalization
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

dataset = MNIST(root='./data', download=True, transform=transform)
BATCH_SIZE = int(len(dataset)/10) # Big but not too big, because I think that the batch is duplicated for each model in the vmap-enselmble
dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, num_workers = 2)

# criterion functional
criterion = F.cross_entropy

# parameter subspaces to investigate.
# for each string, the union of all modules that contain the string in their name will be considered.
# we will sample a random direction in that subspace + the gradient ascent and descent directions projected onto that subspace.
SUBSPACES = ['fc1', 'weight', 'bias']

#check if we are on GPU, otherwise just proof of concept
if torch.cuda.is_available():
    NUM_DIRS = 6
    MAX_OOM = 1
    MIN_OOM = -8
else:
    NUM_DIRS = 0 # number of random directions to sample, in addition to the gradient ascent + descent, and radially in + out.
    MAX_OOM = 0 # maximum order of magnitude to sample
    MIN_OOM = -1 # minimum order of magnitude to sample

Define utility functions for parameter space arithmetic. iadd is inplace, all others create a new model.

In [16]:
from copy import deepcopy

@torch.no_grad
def iadd(a:nn.Module, b:nn.Module):
    """add the parameters of b to a, inplace"""
    for a_param, b_param in zip(a.parameters(), b.parameters()):
        a_param.data.add_(b_param.data)
    return a

@torch.no_grad
def add(a_old:nn.Module, b:nn.Module):
    """add the parameters of b to a"""
    a = deepcopy(a_old)
    return iadd(a, b)

@torch.no_grad
def scale(a_old:nn.Module, b:float):
    """scale the parameters of a by b"""
    a = deepcopy(a_old)
    for a_param in a.parameters():
        a_param.data.mul_(b)
    return a

@torch.no_grad
def sub(a:nn.Module, b:nn.Module):
    """subtract the parameters of b from a"""
    neg_b = scale(b, -1)
    return iadd(neg_b, a)

@torch.no_grad
def abs(a:nn.Module):
    """return the norm of the parameters of a"""
    return torch.norm(torch.cat([param.data.flatten() for param in a.parameters()]))

@torch.no_grad
def norm(a:nn.Module):
    """normalize the parameters of a"""
    return scale(a, 1/abs(a))

@torch.no_grad
def rand_like(a: nn.Module):
    """normalised random direction in the parameter space of the model."""
    rand = deepcopy(a)
    for param in rand.parameters():
        param.data = torch.randn_like(param.data)
    return norm(rand)

@torch.no_grad
def project_to_module(a: nn.Module, target_subspace: str):
    """
    normalised projection onto the subspace of the model specified by `target_subspace`.
    subspace: Any parameter whose name contains the `target_subspace` string.
    """
    projection = deepcopy(a)
    for name, param in projection.named_parameters():
        if target_subspace not in name:
            param.data.zero_()
    return norm(projection)

Define profiling context manager: measure execution time and GPU RAM usage before and after.

In [17]:
from time import perf_counter
from contextlib import contextmanager
import torch

@contextmanager
def profiler(description, length=80, pad_char=':'):
    print('\n'+description.center(length, pad_char))
    if torch.cuda.is_available():
        print(f'GPU RAM before execution: Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    start = perf_counter()
    yield
    if torch.cuda.is_available():
        print(f'GPU RAM after execution:  Allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB | Reserved: {torch.cuda.memory_reserved()/1e9:.2f} GB')
    print(f'Finished {description} in {perf_counter() - start:.2f} s'.center(length, pad_char))

with profiler('Profiler example'):
    print('Computation goes here...')


::::::::::::::::::::::::::::::::Profiler example::::::::::::::::::::::::::::::::
Computation goes here...
::::::::::::::::::::::Finished Profiler example in 0.00 s:::::::::::::::::::::::


In [18]:
center_model = SimpleMLP()

# create a model to find the gradient
grad_model = deepcopy(center_model)
grad_model = grad_model.to(device)

optimizer = torch.optim.SGD(grad_model.parameters(), lr=1)
optimizer.zero_grad()
torch.set_grad_enabled(True)

# take a single step of gradient descent
# gradient accumulate over the whole dataset
with profiler('1 step of gradient descent over the whole dataset'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        output = grad_model(data)
        loss = criterion(output, target)
        loss.backward()

optimizer.step()
# from this point on we wont need any gradients
torch.set_grad_enabled(False)


# calculate the direction of gradient decent from the center model
grad_model = grad_model.to('cpu')
dir_ascent = sub(center_model, grad_model)
dir_ascent = norm(dir_ascent)
dir_descent = scale(dir_ascent, -1)

del grad_model



:::::::::::::::1 step of gradient descent over the whole dataset::::::::::::::::


:::::Finished 1 step of gradient descent over the whole dataset in 19.39 s::::::


In [19]:
radial = norm(center_model)

dirs = {
    'Ascent': dir_ascent,
    'Descent': dir_descent,
    'Radially Out': radial,
    'Radially In': scale(radial,-1),
    }

# add random directions
dirs.update({f'Random {i}': rand_like(center_model) for i in range(NUM_DIRS)})

# add projections of each dir in dirs onto each SUBSPACE
# Add projections of each direction in dirs onto each subspace
for dir_name, direction in list(dirs.items()):
    for subspace in SUBSPACES:
        projected_dir = project_to_module(direction, subspace)
        new_key = f'{dir_name} ⋅ P({subspace})'
        dirs[new_key] = projected_dir

# check that they are all normalised
assert all(torch.isclose(abs(d), torch.tensor(1.0)) for d in dirs.values())

In [20]:
import pandas as pd
import numpy as np

# convert to base 2
POW_MIN_DIST = int(np.floor(MIN_OOM*np.log2(10)))
POW_MAX_DIST = int(np.ceil(MAX_OOM*np.log2(10)))

# will sample these distances
scales = 2. ** np.arange(POW_MIN_DIST, POW_MAX_DIST + 1)

# offset each direction by a different amount, so that we get a nice spread of distances, to avoid artefacts/robustness
offset_per_direction = 2. ** np.linspace(0, 1, len(dirs), endpoint=False)

# Initialize an empty list to hold tuples for MultiIndex
index_list = []

# Calculate distances using broadcasting
distances = np.outer(scales, offset_per_direction).T.flatten()

# Repeat directions accordingly
directions = np.repeat(list(dirs.keys()), len(scales))

# Create DataFrame directly from numpy arrays
df = pd.DataFrame(index=pd.MultiIndex.from_arrays([directions, distances], names=['Direction', 'Distance']))

df


Empty DataFrame
Columns: []
Index: [(Ascent, 0.0625), (Ascent, 0.125), (Ascent, 0.25), (Ascent, 0.5), (Ascent, 1.0), (Descent, 0.06526711140171336), (Descent, 0.13053422280342672), (Descent, 0.26106844560685344), (Descent, 0.5221368912137069), (Descent, 1.0442737824274138), (Radially Out, 0.0681567332915786), (Radially Out, 0.1363134665831572), (Radially Out, 0.2726269331663144), (Radially Out, 0.5452538663326288), (Radially Out, 1.0905077326652577), (Radially In, 0.07117428967229322), (Radially In, 0.14234857934458645), (Radially In, 0.2846971586891729), (Radially In, 0.5693943173783458), (Radially In, 1.1387886347566916), (Ascent ⋅ P(fc1), 0.07432544468767006), (Ascent ⋅ P(fc1), 0.14865088937534013), (Ascent ⋅ P(fc1), 0.29730177875068026), (Ascent ⋅ P(fc1), 0.5946035575013605), (Ascent ⋅ P(fc1), 1.189207115002721), (Ascent ⋅ P(weight), 0.07761611325459275), (Ascent ⋅ P(weight), 0.1552322265091855), (Ascent ⋅ P(weight), 0.310464453018371), (Ascent ⋅ P(weight), 0.620928906036742), (Ascent ⋅ P(weight), 1.241857812073484), (Ascent ⋅ P(bias), 0.0810524721656881), (Ascent ⋅ P(bias), 0.1621049443313762), (Ascent ⋅ P(bias), 0.3242098886627524), (Ascent ⋅ P(bias), 0.6484197773255048), (Ascent ⋅ P(bias), 1.2968395546510096), (Descent ⋅ P(fc1), 0.08464097168355579), (Descent ⋅ P(fc1), 0.16928194336711158), (Descent ⋅ P(fc1), 0.33856388673422316), (Descent ⋅ P(fc1), 0.6771277734684463), (Descent ⋅ P(fc1), 1.3542555469368927), (Descent ⋅ P(weight), 0.08838834764831845), (Descent ⋅ P(weight), 0.1767766952966369), (Descent ⋅ P(weight), 0.3535533905932738), (Descent ⋅ P(weight), 0.7071067811865476), (Descent ⋅ P(weight), 1.4142135623730951), (Descent ⋅ P(bias), 0.09230163412121871), (Descent ⋅ P(bias), 0.18460326824243742), (Descent ⋅ P(bias), 0.36920653648487484), (Descent ⋅ P(bias), 0.7384130729697497), (Descent ⋅ P(bias), 1.4768261459394993), (Radially Out ⋅ P(fc1), 0.0963881765879963), (Radially Out ⋅ P(fc1), 0.1927763531759926), (Radially Out ⋅ P(fc1), 0.3855527063519852), (Radially Out ⋅ P(fc1), 0.7711054127039704), (Radially Out ⋅ P(fc1), 1.5422108254079407), (Radially Out ⋅ P(weight), 0.10065564574682839), (Radially Out ⋅ P(weight), 0.20131129149365679), (Radially Out ⋅ P(weight), 0.40262258298731357), (Radially Out ⋅ P(weight), 0.8052451659746271), (Radially Out ⋅ P(weight), 1.6104903319492543), (Radially Out ⋅ P(bias), 0.10511205190671431), (Radially Out ⋅ P(bias), 0.21022410381342863), (Radially Out ⋅ P(bias), 0.42044820762685725), (Radially Out ⋅ P(bias), 0.8408964152537145), (Radially Out ⋅ P(bias), 1.681792830507429), (Radially In ⋅ P(fc1), 0.10976576002333122), (Radially In ⋅ P(fc1), 0.21953152004666243), (Radially In ⋅ P(fc1), 0.43906304009332486), (Radially In ⋅ P(fc1), 0.8781260801866497), (Radially In ⋅ P(fc1), 1.7562521603732995), (Radially In ⋅ P(weight), 0.1146255054005839), (Radially In ⋅ P(weight), 0.2292510108011678), (Radially In ⋅ P(weight), 0.4585020216023356), (Radially In ⋅ P(weight), 0.9170040432046712), (Radially In ⋅ P(weight), 1.8340080864093424), (Radially In ⋅ P(bias), 0.11970041008732171), (Radially In ⋅ P(bias), 0.23940082017464343), (Radially In ⋅ P(bias), 0.47880164034928685), (Radially In ⋅ P(bias), 0.9576032806985737), (Radially In ⋅ P(bias), 1.9152065613971474)]

[80 rows x 0 columns]

In [21]:
# fill the dataframe with the models
for dir_name, dist in df.index:
    shift = scale(dirs[dir_name], dist)     # shift = direction * distance
    location = iadd(shift, center_model)    # location = center + shift
    df.loc[(dir_name, dist), 'Model'] = location

df

Model
Direction             Distance                                                   
Ascent                0.062500  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      0.125000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      0.250000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      0.500000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      1.000000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
...                                                                           ...
Radially In ⋅ P(bias) 0.119700  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      0.239401  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      0.478802  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      0.957603  SimpleMLP(\n  (fc1): Linear(in_features=784, o...
                      1.915207  SimpleMLP(\n  (fc1): Linear(in_features=784, o...

[80 rows x 1 columns]

Parallel evaluation with vmap()


In [22]:
from torch.func import stack_module_state, functional_call
from torch import vmap

# add the center model
ensemble_list = [center_model] + list(df['Model'])

print(f'Ensemble size: {len(ensemble_list)}')

# stack to prepare for vmap
params, buffers = stack_module_state(ensemble_list)

with profiler(f'Moving stacked ensemble to {device}'):
    params = {name: tensor.to(device) for name, tensor in params.items()}
    buffers = {name: tensor.to(device) for name, tensor in buffers.items()}

stacked_ensemble = (params, buffers)

# Construct a "stateless" version of one of the models. It is "stateless" in
# the sense that the parameters are meta Tensors and do not have storage.
meta_model = deepcopy(center_model).to('meta')

def meta_model_loss(params_and_buffers, data, target):
    """Compute the loss of a set of params on a batch of data, via the meta model"""
    predictions = functional_call(meta_model, params_and_buffers, (data,))
    predictions = predictions.double() # double precision for better resoltion of small scales
    loss = criterion(predictions, target)
    return loss

# define a loss function that takes the stacked ensemble params, data, and target
# in_dims=(0, None, None) adds an ensemble dimension to the first argument 'params_and_buffers' but not to the other two arguments
vmap_loss = vmap(meta_model_loss, in_dims=(0, None, None))

batch_losses = []
with profiler(f'Evaluating stacked ensemble on {device}'):
    for data, target in dataloader:
        data, target = data.to(device), target.to(device)
        batch_loss = vmap_loss(stacked_ensemble, data, target)
        batch_losses.append(batch_loss)

with profiler('Freeing data and target'):
    del data, target
    torch.cuda.empty_cache()

with profiler('Freeing stacked ensemble'):
    del stacked_ensemble
    torch.cuda.empty_cache()

loss_tensor = torch.mean(torch.stack(batch_losses), dim=0)


:::::::::::::::::::::::::Moving stacked ensemble to cpu:::::::::::::::::::::::::
:::::::::::::::Finished Moving stacked ensemble to cpu in 0.00 s::::::::::::::::

:::::::::::::::::::::::Evaluating stacked ensemble on cpu:::::::::::::::::::::::
:::::::::::::Finished Evaluating stacked ensemble on cpu in 57.33 s:::::::::::::

::::::::::::::::::::::::::::Freeing data and target:::::::::::::::::::::::::::::
:::::::::::::::::::Finished Freeing data and target in 0.00 s:::::::::::::::::::

::::::::::::::::::::::::::::Freeing stacked ensemble::::::::::::::::::::::::::::
::::::::::::::::::Finished Freeing stacked ensemble in 0.00 s:::::::::::::::::::


In [23]:
# Convert the loss tensor to a list and unpack
center_loss, *dir_losses = loss_tensor.tolist()

# Ensure the length of dir_losses matches the DataFrame length
assert len(dir_losses) == len(df)

# Add the directional losses to the DataFrame
df['Loss'] = dir_losses

# Add the center_loss to each Direction, for Distance=0
for direction in df.index.get_level_values('Direction').unique():
    df.loc[(direction, 0.0), 'Loss'] = center_loss

df.sort_index(inplace=True)

df

Model  \
Direction                Distance                                                      
Ascent                   0.000000                                                NaN   
                         0.062500  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.125000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.250000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.500000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
...                                                                              ...   
Radially Out ⋅ P(weight) 0.100656  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.201311  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.402623  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.805245  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         1.610490  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   

                                       Loss  
Direction                Distance            
Ascent                   0.000000  2.303077  
                         0.062500  2.335889  
                         0.125000  2.368436  
                         0.250000  2.430218  
                         0.500000  2.538586  
...                                     ...  
Radially Out ⋅ P(weight) 0.100656  2.303110  
                         0.201311  2.303151  
                         0.402623  2.303259  
                         0.805245  2.303591  
                         1.610490  2.304842  

[96 rows x 2 columns]

In [24]:
import plotly.graph_objects as go

# Create a Plotly figure
fig = go.Figure()

# Plotting directly from grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Loss'],
        mode='lines+markers',
        name=direction
    ))

# Set axis labels and title with adjusted figure dimensions
fig.update_layout(
    title=f'Loss Landscape of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    xaxis_title='Distance from Center in Parameter Space',
    yaxis_title='Loss',
    legend_title='Direction',
    template='seaborn',
    height=600,
)


# Show the figure
fig.show()


Now let's measure the roughness.

In [25]:
# TODO: Add nice explanation of the roughness calculation
df['Roughness'] = np.nan

# Calculate the roughness, one direction at a time
for direction, group in df.groupby(level='Direction'):
    distA = group.index.get_level_values('Distance').to_numpy()
    losses = group['Loss'].to_numpy()

    # Named slices, so that 
    # distA[x] really corresponds to the distance between A and X ∈ {A, B, C},
    # losses[x] really corresponds to the loss at X ∈ {A, B, C}
    a = 0
    b = slice(1, -1)
    c = slice(2, None)

    # Ensure distances and slices are correct
    assert np.all(distA[c] == 2 * distA[b]), "Distance check failed"

    # Calculate paths between A, B, and C
    AB = np.sqrt((losses[b] - losses[a]) ** 2 + distA[b] ** 2)
    BC = np.sqrt((losses[c] - losses[b]) ** 2 + distA[b] ** 2)
    AC = np.sqrt((losses[c] - losses[a]) ** 2 + distA[c] ** 2)

    # Roghness is how much the path via B (A->B->C) is longer than the direct path A->C
    # subtracting 1, then we can use log scale better!
    roughness = (AB + BC) / AC - 1

    # Update DataFrame with roughness values
    df.loc[(direction,), 'Roughness'] = np.concatenate([[np.nan], roughness, [np.nan]])

df

Model  \
Direction                Distance                                                      
Ascent                   0.000000                                                NaN   
                         0.062500  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.125000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.250000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.500000  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
...                                                                              ...   
Radially Out ⋅ P(weight) 0.100656  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.201311  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.402623  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         0.805245  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   
                         1.610490  SimpleMLP(\n  (fc1): Linear(in_features=784, o...   

                                       Loss  Roughness  
Direction                Distance                       
Ascent                   0.000000  2.303077        NaN  
                         0.062500  2.335889   1.000001  
                         0.125000  2.368436   1.000065  
                         0.250000  2.430218   1.000472  
                         0.500000  2.538586   1.000280  
...                                     ...        ...  
Radially Out ⋅ P(weight) 0.100656  2.303110   1.000000  
                         0.201311  2.303151   1.000000  
                         0.402623  2.303259   1.000000  
                         0.805245  2.303591   1.000000  
                         1.610490  2.304842        NaN  

[96 rows x 3 columns]

In [29]:
# new nomenclature: grit := scale dependent roughness

# Create a Plotly figure
fig = go.Figure()

# Plotting roughness for each direction using grouped data
for direction, data in df.groupby(level='Direction'):
    fig.add_trace(go.Scatter(
        x=data.index.get_level_values('Distance'),
        y=data['Roughness'],
        mode='markers',
        name=direction
    ))

# Set axis labels, title, and configure log scale on x-axis
fig.update_layout(
    title=f'Scale Dependent Roughness of {center_model.__class__.__name__} on {dataset.__class__.__name__}',
    xaxis={
        'title': 'Coarse Graining Scale',
        'type': 'log',
        'dtick': 1,  # Tick every power of ten
    },
    yaxis ={
        'title': 'Roughness',
        'type': 'log',
    },
    legend_title='Direction',
    template='seaborn',
    height=600
)

# Display the figure
fig.show()
